In [ ]:
!pip install kaggle

In [8]:
import time
import torch
import zipfile
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from google.colab import files
from sklearn.ensemble import RandomForestClassifier
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report, accuracy_score, log_loss

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nadavtoledo1","key":"c2c6f0bc23c634022c5d4c2c13e88490"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder


Dataset URL: https://www.kaggle.com/datasets/jutrera/stanford-car-dataset-by-classes-folder
License(s): other
100% 1.82G/1.83G [00:15<00:00, 192MB/s]
100% 1.83G/1.83G [00:15<00:00, 128MB/s]


In [5]:
with zipfile.ZipFile("stanford-car-dataset-by-classes-folder.zip", "r") as zip_ref:
    zip_ref.extractall("stanford_cars")

In [6]:
# Define the transformation for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the path to the dataset
data_dir = '/content/stanford_cars/car_data/car_data/train'

# Load the full dataset using ImageFolder
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Define the split ratio: e.g., 80% for training and 20% for validation
train_size = int(0.8 * len(dataset))  # 80% for training
valid_size = len(dataset) - train_size  # 20% for validation

# Split the dataset into train and validation sets
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# Create DataLoader for both training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_dataset = datasets.ImageFolder(root='/content/stanford_cars/car_data/car_data/test', transform=transform)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
# Function to count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Function to evaluate a model (both validation and test)
def evaluate_model(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    unique_correct = set()
    unique_errors = set()

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Predictions
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Track unique correct and errors
            for i, label in enumerate(labels):
                if predicted[i] == label:
                    unique_correct.add((predicted[i].item(), label.item()))
                else:
                    unique_errors.add((predicted[i].item(), label.item()))

    avg_loss = running_loss / len(loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy, unique_correct, unique_errors

# Training loop
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10):
    best_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Print training loss for each epoch
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

        # Validate after each epoch
        valid_loss, valid_accuracy, _, _ = evaluate_model(model, valid_loader, criterion)
        print(f'Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%')

        # Check if validation accuracy improved
        if valid_accuracy > best_accuracy:
            best_accuracy = valid_accuracy
            torch.save(model.state_dict(), 'best_model.pth')  # Save the best model

    print("Training complete.")
    return model

# ResNet

In [ ]:
# Initialize model (e.g., ResNet-50)
ResNetmodel = models.resnet50(pretrained=True)
num_classes = 196  # Stanford Cars has 196 classes
ResNetmodel.fc = nn.Linear(ResNetmodel.fc.in_features, num_classes)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ResNetmodel.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(ResNetmodel.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()


# Run the training
ResNetmodel = train_model(ResNetmodel, train_loader, valid_loader, criterion, optimizer)

# Load best model
ResNetmodel.load_state_dict(torch.load('best_model.pth'))

# Final evaluation on validation and test set
valid_loss, valid_accuracy, valid_unique_correct, valid_unique_errors = evaluate_model(ResNetmodel, valid_loader, criterion)
test_loss, test_accuracy, test_unique_correct, test_unique_errors = evaluate_model(ResNetmodel, test_loader, criterion)

# Print final results
print(f'\nValidation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Get number of parameters
num_params = count_parameters(ResNetmodel)

results = pd.DataFrame(columns=['Model Name', '# Parameters', 'Validation Loss', 'Validation Accuracy (%)', 'Test Loss', 'Test Accuracy (%)', '# Unique Correct Samples', '# Unique Errors'])
results.loc[len(results)] = ['ResNet-50', num_params, valid_loss, valid_accuracy, test_loss, test_accuracy, len(valid_unique_correct), len(valid_unique_errors)]

print(f'\nModel Name: ResNet-50')
print(f'# Parameters: {num_params}')
print(f'Validation Loss: {valid_loss:.4f}')
print(f'Validation Accuracy: {valid_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.2f}%')
print(f'# Unique Correct Samples: {len(valid_unique_correct)}')
print(f'# Unique Errors: {len(valid_unique_errors)}')



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 82.9MB/s]


# VGG-16

In [ ]:
# Load VGG-16 with pretrained weights
VGG16model = models.vgg16(pretrained=True)

VGG16model.classifier[6] = nn.Linear(VGG16model.classifier[6].in_features, num_classes)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
VGG16model.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(VGG16model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Run the training
VGG16model = train_model(VGG16model, train_loader, valid_loader, criterion, optimizer)

# Load best model
VGG16model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation on validation and test set
valid_loss, valid_accuracy, valid_unique_correct, valid_unique_errors = evaluate_model(VGG16model, valid_loader, criterion)
test_loss, test_accuracy, test_unique_correct, test_unique_errors = evaluate_model(VGG16model, test_loader, criterion)

# Print final results
print(f'\nValidation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Get number of parameters
num_params = count_parameters(VGG16model)

results.loc[len(results)] = ['VGG-16', num_params, valid_loss, valid_accuracy, test_loss, test_accuracy, len(valid_unique_correct), len(valid_unique_errors)]

print(f'\nModel Name: VGG-16')
print(f'# Parameters: {num_params}')
print(f'Validation Loss: {valid_loss:.4f}')
print(f'Validation Accuracy: {valid_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.2f}%')
print(f'# Unique Correct Samples: {len(valid_unique_correct)}')
print(f'# Unique Errors: {len(valid_unique_errors)}')

# DenseNet-121

In [ ]:
# Load DenseNet-121 with pretrained weights
DenseNet121model = models.densenet121(pretrained=True)
DenseNet121model.classifier = nn.Linear(DenseNet121model.classifier.in_features, num_classes)
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DenseNet121model.to(device)
# Define optimizer and loss function
optimizer = optim.Adam(DenseNet121model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
# Run the training
DenseNet121model = train_model(DenseNet121model, train_loader, valid_loader, criterion, optimizer)

# Load best model
DenseNet121model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation on validation and test set
valid_loss, valid_accuracy, valid_unique_correct, valid_unique_errors = evaluate_model(DenseNet121model, valid_loader, criterion)
test_loss, test_accuracy, test_unique_correct, test_unique_errors = evaluate_model(DenseNet121model, test_loader, criterion)

# Print final results
print(f'\nValidation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Get number of parameters
num_params = count_parameters(DenseNet121model)

results.loc[len(results)] = ['DenseNet-121', num_params, valid_loss, valid_accuracy, test_loss, test_accuracy, len(valid_unique_correct), len(valid_unique_errors)]
print(f'\nModel Name: DenseNet-121')
print(f'# Parameters: {num_params}')
print(f'Validation Loss: {valid_loss:.4f}')
print(f'Validation Accuracy: {valid_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.2f}%')
print(f'# Unique Correct Samples: {len(valid_unique_correct)}')
print(f'# Unique Errors: {len(valid_unique_errors)}')

# EfficientNet-B0

In [ ]:
# Load EfficientNet-B0 with pretrained weights
EfficientNetB0model = models.efficientnet_b0(pretrained=True)
EfficientNetB0model.classifier[1] = nn.Linear(EfficientNetB0model.classifier[1].in_features, num_classes)
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EfficientNetB0model.to(device)
# Define optimizer and loss function
optimizer = optim.Adam(EfficientNetB0model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
# Run the training
EfficientNetB0model = train_model(EfficientNetB0model, train_loader, valid_loader, criterion, optimizer)

# Load best model
EfficientNetB0model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation on validation and test set
valid_loss, valid_accuracy, valid_unique_correct, valid_unique_errors = evaluate_model(EfficientNetB0model, valid_loader, criterion)
test_loss, test_accuracy, test_unique_correct, test_unique_errors = evaluate_model(EfficientNetB0model, test_loader, criterion)

# Print final results
print(f'\nValidation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Get number of parameters
num_params = count_parameters(EfficientNetB0model)

results.loc[len(results)] = ['EfficientNet-B0', num_params, valid_loss, valid_accuracy, test_loss, test_accuracy, len(valid_unique_correct), len(valid_unique_errors)]

print(f'\nModel Name: EfficientNet-B0')
print(f'# Parameters: {num_params}')
print(f'Validation Loss: {valid_loss:.4f}')
print(f'Validation Accuracy: {valid_accuracy:.2f}%')
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.2f}%')
print(f'# Unique Correct Samples: {len(valid_unique_correct)}')
print(f'# Unique Errors: {len(valid_unique_errors)}')

In [ ]:
# Data for the models
data = {
    "Model Name": ["ResNet-50", "VGG-16", "DenseNet-121", "EfficientNet-B0"],
    "# Parameters": [23909636, 135063556, 7154756, 4258624],
    "Validation Loss": [0.9400, 1.7591, 1.3922, 1.5058],
    "Validation Accuracy (%)": [75.45, 52.73, 73.48, 64.33],
    "Test Loss": [0.8789, 1.7204, 1.3844, 1.4672],
    "Test Accuracy (%)": [77.02, 53.41, 74.27, 67.14],
    "# Unique Correct Samples": [190, 192, 194, 192],
    "# Unique Errors": [297, 578, 332, 461],
}

print("5 Epoches ____________________________________________________")
# Create the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

print("10 Epoches ____________________________________________________")
print(results)


5 Epoches ____________________________________________________
        Model Name  # Parameters  Validation Loss  Validation Accuracy (%)  \
0        ResNet-50      23909636           0.9400                    75.45   
1           VGG-16     135063556           1.7591                    52.73   
2     DenseNet-121       7154756           1.3922                    73.48   
3  EfficientNet-B0       4258624           1.5058                    64.33   

   Test Loss  Test Accuracy (%)  # Unique Correct Samples  # Unique Errors  
0     0.8789              77.02                       190              297  
1     1.7204              53.41                       192              578  
2     1.3844              74.27                       194              332  
3     1.4672              67.14                       192              461  
10 Epoches ____________________________________________________


NameError: name 'results' is not defined

# d.

In [8]:
# Load the pretrained ResNet-50 model
model = models.resnet50(pretrained=True)

# Remove the last layer (fc layer) to use it as a feature extractor
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

# Move the model to the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
feature_extractor.to(device)
feature_extractor.eval()

# Function to extract features
def extract_features(loader):
    features = []
    labels = []
    with torch.no_grad():
        for images, lbls in loader:
            images = images.to(device)
            output = feature_extractor(images)
            output = output.view(output.size(0), -1)  # Flatten the features
            features.append(output.cpu().numpy())
            labels.append(lbls.numpy())
    return np.vstack(features), np.hstack(labels)

# Extract features for the train and validation datasets
train_features, train_labels = extract_features(train_loader)
valid_features, valid_labels = extract_features(valid_loader)

# Split the extracted features into train and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
    train_features, train_labels, test_size=0.2, random_state=42
)

# Train a Random Forest classifier on the extracted features
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Validate the classifier
y_pred = rf_classifier.predict(X_valid)

# Evaluate the performance
print("Validation Results using Feature Extractor + Random Forest:")
print(classification_report(y_valid, y_pred))

# Test the model on the validation set
valid_predictions = rf_classifier.predict(valid_features)
valid_accuracy = accuracy_score(valid_labels, valid_predictions)
print(f"Validation Accuracy (Feature Extractor + RF): {valid_accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 129MB/s]


Validation Results using Feature Extractor + Random Forest:
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         8
           1       0.11      0.40      0.17         5
           2       0.00      0.00      0.00         2
           3       0.20      0.11      0.14         9
           4       0.13      0.29      0.18         7
           5       0.00      0.00      0.00         7
           6       0.25      0.33      0.29         9
           7       0.20      0.10      0.13        10
           8       0.25      0.33      0.29         6
           9       0.29      0.67      0.40         3
          10       0.11      0.20      0.14         5
          11       0.12      0.25      0.17         8
          12       0.50      0.75      0.60         4
          13       0.17      0.40      0.24         5
          14       0.09      0.25      0.13         4
          15       0.00      0.00      0.00         5
          16       0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
# Data Transformations
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}


# Load Pretrained Model (Feature Extractor)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
num_parameters = sum(p.numel() for p in model.parameters())
model = model.to(device)
model.eval()

# Remove the classifier (fc layer)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

# Feature Extraction Function
def extract_features(loader):
    features, labels = [], []
    with torch.no_grad():
        for inputs, targets in loader:
            inputs = inputs.to(device)
            outputs = feature_extractor(inputs)
            outputs = outputs.view(outputs.size(0), -1).cpu().numpy()
            features.append(outputs)
            labels.append(targets.numpy())
    return np.vstack(features), np.hstack(labels)

# Extract Features for Train, Validation, and Test Sets
start_time = time.time()
X_train, y_train = extract_features(train_loader)
X_val, y_val = extract_features(valid_loader)
X_test, y_test = extract_features(test_loader)
feature_extraction_time = time.time() - start_time

# Train Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make Predictions and Compute Metrics
y_val_pred = rf.predict(X_val)
y_val_proba = rf.predict_proba(X_val)
y_test_pred = rf.predict(X_test)
y_test_proba = rf.predict_proba(X_test)

# Validation Metrics
val_loss = log_loss(y_val, y_val_proba)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Test Metrics
test_loss = log_loss(y_test, y_test_proba)
test_accuracy = accuracy_score(y_test, y_test_pred)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 186MB/s]


Feature Extraction Runtime (s): 169.04
Validation Loss: 7.6355
Validation Accuracy (%): 21.18
Test Loss: 7.0027
Test Accuracy (%): 23.39
# Parameters in Feature Extractor: 25557032


In [10]:
# Print Results
print("Feature Exractor with Random Forest")
print(f"Runtime (s): {feature_extraction_time:.2f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy (%): {val_accuracy * 100:.2f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy (%): {test_accuracy * 100:.2f}")
print(f"# Parameters in Feature Extractor: {num_parameters}")

Feature Exractor with Random Forest
Runtime (s): 169.04
Validation Loss: 7.6355
Validation Accuracy (%): 21.18
Test Loss: 7.0027
Test Accuracy (%): 23.39
# Parameters in Feature Extractor: 25557032
